# Segmentation tasks

The following notebook performs the semantic segmentation task on 4 sets of images:
- raw images
- noisy augmented images
- denoised augmented images
- denoised raw images

It will make use of a list of models and compute metrics on them.

In [1]:
%load_ext autoreload
%autoreload 2

from models_loading import *
from data_loading import * 
from tasks_loading import *
from metrics_loading import *
from time_loading import *

from utils import *

# Used for tests purpose
import torch

In [2]:
start = start_timer()

## Models

### Models loading

In [3]:
load_model("", list_available=True)

# sinet = load_model("SINet").eval()
unet = load_model("UNet").eval()

Available models aliases:
  SINet
  UNet
  HRNet


In [4]:
# Fake data
x = torch.randn(1, 3, 1024, 2048)

In [5]:
# y = sinet(x)
# print(y.shape)
# y = unet(x)
# print(y.shape)

### Data Loading

In [6]:
# Set here the directories locations to the data for the evaluation task

cityscapes_dir = 'HRNet/data/cityscapes'
denoized_dir = '../candidate_model_predictions'
# cityscapes_distorted_dir = 'HRNet/data/cityscapes'

In [7]:
load_data("", list_available=True)

# cs = load_data("cityscapes", location=cityscapes_dir)

Available dataloaders aliases:
  cityscapes


### Evaluation Task

In [8]:
load_tasks("", list_available=True)

semantic_task = load_tasks("semantic")
semantic_unet = semantic_task(unet)

Available tasks aliases:
  semantic


In [9]:
y = semantic_unet(x)
y.shape

Using 19 classes.


torch.Size([1, 19, 1024, 2048])

In [10]:
# y = semantic_unet(x)
y, colored = semantic_unet(x, get_colored=True)
y, classified = semantic_unet(x, get_classification=True)
y.shape, classified.shape, colored.shape

Using 19 classes.
Using 19 classes.


(torch.Size([1, 19, 1024, 2048]), (1, 1024, 2048), (1, 3, 1024, 2048))

### Metrics

In [11]:
load_metrics("", list_available=True)

iou = load_metrics("iou")
dice = load_metrics("dice")

Available tasks aliases:
  iou
  dice


In [12]:
print(dice(y, torch.from_numpy(classified)))
print(iou(y, torch.from_numpy(classified)))

tensor(1.)
tensor(1.)


In [13]:
store_results(colored, 'output/')

(1, 1024, 2048, 3)
(1, 1024, 2048, 3)
output/result_0.png


In [14]:
a = defaultdict(dict)
a['test']

{}

In [18]:
# stopping timer
end = end_timer()
time_spent = time_for_report(start, end)


In [19]:
report = Report()

report.add_dataset("Cityscapes")
report.add_task_model('UNet', 'semantic')
report.add_task_model('SINet', 'semantic')

report.add_metric('dice', '98.12%', "RAW")
report.add_metric('dice', '98.12%', "DENOISED")
report.add_metric('dice', '98.12%', "RAW_NOISY")
report.add_metric('dice', '98.12%', "NOISY_DENOISED")

report.add_time_spent(time_spent)

report.add_metric('iou', '12.12%', 'RAW')

print(report.generate(path='reports/', as_string=True))

###### REPORT

#### Datasets used
Cityscapes

#### Tasks
semantic

#### Tasks models used
UNet                 [semantic]
SINet                [semantic]

#### Time spent
0days-00:01:34.03

#### Metrics Results
## dice
RAW                 98.12%
DENOISED            98.12%
RAW_NOISY           98.12%
NOISY_DENOISED      98.12%

## iou
RAW                 12.12%



